# Intro to Dynamic Programming
**Overview.** This lecture covers basic concepts from Dynamic Programming. The exposition follows Adda and Cooper (2003).<sup>1</sup> A more in depth treatment can be found in Stokey and Lucas (1989).<sup>2</sup>

## The Bellman Equation
Let $\tilde\sigma(s_t,c_t)$ denote the payoff to an agent. $s_t$ is a state vector which is a set of variables that affect payoffs but that the agent has no influence over contemporaneously. The agent can choose the control $c_t$ to influence the *evolution* of the state through the transition equation $s_{t+1} = \tau(s_t,c_t)$. Letting $V(s_0)$ denote the maximized value of
$$\sum_{t=0}^\infty\beta^t\tilde\sigma(s_t,c_t)$$
The principal of optimality is concisely stated in the *Bellman Equation*
$$V(s) = \max_{c\in C(s)}\left\{\tilde\sigma(s,c) + \beta V(s')\right\}\quad\text{all } s\in S$$
such that $s' = \tau(s,c)$. Note that the set of choices for the control may depend on $s$. One can write this more compactly as
$$V(s) = \max_{s'\in \Gamma(s)}\left\{\sigma(s,s') + \beta V(s')\right\}\quad\text{all } s\in S$$
where the control is the future state.
## Two Theorems

## References
1. Adda, J., & Cooper, R. W. (2003). Dynamic Economics: Quantitative Methods and Applications. MIT Press.
2. Stokey, N. L., & Lucas Jr, R. E. (1989). Recursive Methods in Economic Dynamics. Harvard University Press.

# Ex. The Cake Eating Problem